# Klasifikasi Jenis Beras

# **Mengambil Dataset dari Google Grive**

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!unzip '/content/drive/My Drive/Klasifikasi Beras Satuan.zip'

# **Mengambil Dataset dari Github**

In [0]:
!apt-get install subversion > /dev/null

!svn export https://github.com/Vidi005/Klasifikasi-Beras-Satuan/trunk/data > /dev/null
!svn export https://github.com/Vidi005/Klasifikasi-Beras-Satuan/trunk/utils > /dev/null
!svn export https://github.com/Vidi005/Klasifikasi-Beras-Satuan/trunk/Klasifikasi-Beras-Satuan.ipynb > /dev/null

## Persiapan

How will you classify the bellow two images?

![1rice](https://github.com/totti0223/deep_learning_for_biologists_with_keras/raw/master/assets/11.jpg) Ukuran beras pendek

![2rice](https://github.com/totti0223/deep_learning_for_biologists_with_keras/raw/master/assets/141.jpg) Ukuran beras sedang


In this tutorial, we will construct a classifier in 3 ways, giving you a rough implementation of how deep learning (step 3) can ignore the manual feature extraction we had to do (step 1 and 2). Although you may notice that the classification task is two easy that step 1 or step 2 is sufficient, will compare all steps for understanding.

Step 1. Manual Classsification by Conventional Approach

Step 2. Support Vector Machine based Classification

Step 3. Convolutional Neural Network based Classification

click the run button in each cell from the top to bottom and everything should run fine.
if you don't want to see the results beforehand, click "edit" and select "delete all the output of the cells"


## Memasukan Fungsi Library

In [0]:
import numpy as np 
import math, os, sys
import itertools

import matplotlib.pyplot as plt
plt.style.use('default')
from scipy import ndimage

from skimage import measure, morphology
from skimage.io import imsave, imread
from skimage.color import rgb2gray
from skimage.filters import threshold_otsu
from skimage.transform import resize

from sklearn import svm, datasets
from sklearn.metrics import confusion_matrix
import pandas as pd

In [0]:
#functions to assist visualizations

#confusion matrix drawing function provided by sklearn
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    #code from https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html#sphx-glr-auto-examples-model-selection-plot-confusion-matrix-py
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    #plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()


def make_meshgrid(x, y, h=.02):
    """Create a mesh of points to plot in

    Parameters
    ----------
    x: data to base x-axis meshgrid on
    y: data to base y-axis meshgrid on
    h: stepsize for meshgrid, optional

    Returns
    -------
    xx, yy : ndarray
    #code from https://scikit-learn.org/stable/auto_examples/svm/plot_iris.html#sphx-glr-auto-examples-svm-plot-iris-py
    """
    x_min, x_max = x.min() - 1, x.max() + 1
    y_min, y_max = y.min() - 1, y.max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    return xx, yy


def plot_contours(clf, xx, yy, **params):
    """Plot the decision boundaries for a classifier.

    Parameters
    ----------
    ax: matplotlib axes object
    clf: a classifier
    xx: meshgrid ndarray
    yy: meshgrid ndarray
    params: dictionary of params to pass to contourf, optional
    """
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    out = plt.contourf(xx, yy, Z, **params)
    return out

## Mengimpor Dataset

In [0]:
#this command below see inspects part of the files which have been downloaded from the cell above
!ls data/image
!ls data/image/train
!ls data/image/train/pendek
!ls data/image/train/sedang

In [0]:
#let's visualize a single file
image = imread("data/image/train/pendek/C1.jpg")
plt.figure(figsize=(3,3))
plt.imshow(image)

In [0]:
#lets load everything into memory first

#load training dataset
X_train = []
y_train = []

for root, dirs, files in os.walk("data/image/train"):
    files = [x for x in files if x.endswith(".jpg")]
    for file in files:
        image_path = os.path.join(root, file)
        
        image = imread(image_path)/255.
        image = resize(image,(32,32))
        X_train.append(image)        
        category = os.path.split(root)[-1]
        if category == "pendek":
            y_train.append(0)
        else:
            y_train.append(1)
X_train = np.array(X_train)
y_train = np.array(y_train)



#load test dataset
X_test = []
y_test = []

for root, dirs, files in os.walk("data/image/test"):
    files = [x for x in files if x.endswith(".jpg")]
    for file in files:
        image_path = os.path.join(root, file)
        
        image = imread(image_path)/255.
        image = resize(image,(32,32))
        X_test.append(image)        
        category = os.path.split(root)[-1]
        if category == "pendek":
            y_test.append(0)
        else:
            y_test.append(1)
X_test = np.array(X_test)
y_test = np.array(y_test)

print("train dataset shape is:", X_train.shape,y_train.shape)
print("test dataset shape is:", X_test.shape,y_test.shape)

In [0]:
plt.subplots_adjust(wspace=0.4, hspace=0.6)

#randomly show several images from the training dataset
index = np.random.randint(0,X_train.shape[0],size=9)

for i, idx  in enumerate(index):
    plt.subplot(3,3,i+1)
    
    if y_train[idx] == 0:
        label = "pendek"
    else:
        label = "sedang"
    plt.title(label)
    plt.imshow(X_train[idx])

# Step 1 Manual Classification

Lets manually define the feature for classification. In this case, simply binarizing the image, extracting the area size of each image and then defining a threshold will do well. All of the image transformation and area measuring can be done with the functions of skimage and scipy.

In [0]:
#Let's try it for one image
image = X_train[0]

#the original image
print(image.shape)
plt.figure(figsize=(3,3))
plt.imshow(image)
plt.title("Gambar ORI")
plt.show()

In [0]:
#gray conversion
gray = rgb2gray(image)
print(gray.shape)
plt.figure(figsize=(3,3))
plt.imshow(gray, cmap=plt.cm.gray)
plt.title("Terkonversi hitam putih")
plt.show()

In [0]:
#binary conversion
threshold = threshold_otsu(gray)
binary = gray > threshold
plt.figure(figsize=(3,3))
plt.imshow(binary, cmap=plt.cm.gray)
plt.show()

now that we have a nice binary image, we can isolate the region of rice seed which corresponds to the white region of the image above.

In [0]:
label_im, nb_labels = ndimage.label(binary)
regionprops = measure.regionprops(label_im, intensity_image=gray)
regionprop = regionprops[0]

print("area is",regionprop.area)
print("major axis length is", regionprop.major_axis_length)
print("minor axis length is", regionprop.minor_axis_length)

In [0]:
#bundling the above into a function
def quantify_area(image):
    gray = rgb2gray(image)
    threshold = threshold_otsu(gray)
    binary = gray > threshold
    label_im, nb_labels = ndimage.label(binary)
    regionprops = measure.regionprops(label_im, intensity_image=gray)
    regionprop = regionprops[0]
    area = regionprop.area
    return area

#test
area = quantify_area(image)
print(area)

In [0]:
X_train_area = []
for image in X_train:
    area = quantify_area(image)
    X_train_area.append(area)

X_test_area = []
for image in X_test:
    area = quantify_area(image)
    X_test_area.append(area)

In [0]:
#check the calculated data area value of training dataset

plt.scatter(range(len(X_train_area)),X_train_area,c=y_train,cmap="jet")
plt.xlabel("gambar beras")
plt.ylabel("ukuran (px)")
plt.show()

In [0]:
#define an area threshold that can seperate the two classes
#must change from the default value or it won't seperate nicely.
#run this code once and try the suitable value that can seperate blue and red with the horizontal lines
area_threshold = 200


#classify whether the image is a proper seed or a broken seed according to the area_threshold value
train_y_pred = []
for area in X_train_area:
    if area > area_threshold:
        train_y_pred.append(0)
    else:
        train_y_pred.append(1)
        

#plot scatter with threshold line
plt.figure(figsize=(5,3))
plt.scatter(range(len(X_train_area)),X_train_area,c=y_train,cmap=plt.cm.coolwarm)
plt.axhline(y=area_threshold)
plt.title("biru: beras pendek, merah: beras sedang")
plt.show()

#calculate confusion matrix
cnf = confusion_matrix(y_train, train_y_pred)

#confusion matrix in figure
plt.figure(figsize=(3,3))
plot_confusion_matrix(cnf, classes=["pendek","sedang"])

plt.show()

In [0]:
#evaluate it with the test dataset
test_y_pred = []
for area in X_test_area:
    if area > area_threshold:
        test_y_pred.append(0)
    else:
        test_y_pred.append(1)
        
#plot scatter with threshold line
plt.figure(figsize=(5,3))
plt.scatter(range(len(X_test_area)),X_test_area,c=y_test,cmap=plt.cm.coolwarm)
plt.axhline(y=area_threshold)
#plt.plot([100,0],[100,350],'k-',lw=2)
plt.show()

#calculate confusion matrix
cnf = confusion_matrix(y_test, test_y_pred)

#confusion matrix in figure
plt.figure(figsize=(3,3))
plot_confusion_matrix(cnf, classes=["pendek","sedang"])
plt.show()

In [0]:
#build a classifier

def manual_classifier(image,area_threshold):
    gray = rgb2gray(image)
    threshold = threshold_otsu(gray)
    binary = gray > threshold
    label_im, nb_labels = ndimage.label(binary)
    regionprops = measure.regionprops(label_im, intensity_image=gray)
    regionprop = regionprops[0]
    area = regionprop.area
    if area > area_threshold:
        return 0
    else:
        return 1

# get a image from test dataset #value must be lower than the size of the test dataset(20-1)
n = 5
image = X_test[n]
label = y_test[n]
area_threshold = 350
prediction = manual_classifier(image,area_threshold)

plt.imshow(image)
print("correct label is: ",label)
print("predicted label is: ",prediction)

# Step 2 Support Vector Machine Classification

We will next use Support Vector Machine (SVM) for Classification. Since SVM uses data greater than two dimension (features), we will add another metrics, major_axis_length of the region of interest.

In [0]:
def quantify_mal(image):
    gray = rgb2gray(image)
    threshold = threshold_otsu(gray)
    binary = gray > threshold
    label_im, nb_labels = ndimage.label(binary)
    regionprops = measure.regionprops(label_im, intensity_image=gray)
    regionprop = regionprops[0]
    mal = regionprop.major_axis_length
    return mal


X_train_mal = []
for image in X_train:
    mal = quantify_mal(image)
    X_train_mal.append(mal)

X_test_mal = []
for image in X_test:
    mal = quantify_mal(image)
    X_test_mal.append(mal)

In [0]:
#reshape data for svm input
X_train2 = np.array([[x,y] for x,y in zip(X_train_mal,X_train_area)])
X_test2 = np.array([[x,y] for x,y in zip(X_test_mal,X_test_area)])
#just concatenating the two data
print(X_train2[:5,:])

In [0]:
#define linear support vector machine for defining the threshold
clf=svm.SVC(kernel="linear")
#train the classifier
clf.fit(X_train2,y_train)


In [0]:
xx, yy = make_meshgrid(X_train2[:,0],X_train2[:,1],h=0.08)
plot_contours(clf, xx, yy,
                  cmap=plt.cm.coolwarm, alpha=0.8)
plt.scatter(X_train2[:,0],X_train2[:,1],c=y_train,cmap=plt.cm.coolwarm)
plt.title("svm classifier for train data")
plt.xlabel("major_axis_length (px)")
plt.ylabel("area (px)")
plt.show()

#Coordinates in the red region are classified broken seeds, while blue as proper shaped seeds

In [0]:
xx, yy = make_meshgrid(X_test2[:,0],X_test2[:,1],h=0.08)
plot_contours(clf, xx, yy,
                  cmap=plt.cm.coolwarm, alpha=0.8)
plt.scatter(X_test2[:,0],X_test2[:,1],c=y_test,cmap=plt.cm.coolwarm)
plt.title("svm classifier for test data")
plt.xlabel("major_axis_length (px)")
plt.ylabel("area (px)")
plt.show()

In [0]:
#construct a classifier function
def svm_classifier(clf,image):
    gray = rgb2gray(image)
    threshold = threshold_otsu(gray)
    binary = gray > threshold
    label_im, nb_labels = ndimage.label(binary)
    regionprops = measure.regionprops(label_im, intensity_image=gray)
    regionprop = regionprops[0]
    area = regionprop.area
    result = clf.predict(np.array([1,area]).reshape(1, -1))
    return result
# get a image from test dataset #value must be lower than the size of the test dataset(20-1)
n = 1
image = X_test[n]
label = y_test[n]
prediction = svm_classifier(clf,image)

plt.imshow(image)
print("correct label is: ",label)
print("predicted label is: ",prediction)

# Step 3 Seperating the two classes with Deep learning (Convolutional Neural Network)

Finaly, we will use Convolutional Neural Network, a type of deep learning architecture that can handle images. By using CNN, we are freed from defining a suitable feature and only have to feed the images to the network. The CNN will find the most suitable features for classification

In [0]:
import keras
from keras import backend as K
from keras.models import Sequential
from keras import layers
from keras.utils.np_utils import to_categorical

from sklearn.model_selection import train_test_split

## Prepare Input Data

In [0]:
y_train2 = to_categorical(y_train)
X_train3, X_valid3, y_train3, y_valid3 = train_test_split(X_train, y_train2, test_size=0.2, random_state=42)

## Model Construction

In [0]:
model = Sequential([
    layers.Conv2D(16, (3,3), input_shape=(32,32,3),name="conv1"),
    layers.Activation("relu"),
    layers.MaxPool2D((2,2),name="pool1"),
    layers.Dropout(0.25),
    layers.Conv2D(32, (3,3),name="conv2"),
    layers.Activation("relu"),
    layers.MaxPool2D((2,2),name="pool2"),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(64,name="fc1"),
    layers.Activation("relu"),
    layers.Dense(2,name="fc2"),
    layers.Activation("softmax")
])

model.compile("adam",loss="categorical_crossentropy",metrics=["acc"])
model.summary()

## Training

In [0]:
history = model.fit(x = X_train3, y = y_train3, batch_size=32, epochs= 10,validation_data=(X_valid3,y_valid3))

## Visualizing Training Result

In [0]:
plt.plot(history.history["acc"],label="train_accuracy")
plt.plot(history.history["val_acc"],label="validation_accuracy")
plt.legend()
plt.show()

plt.plot(history.history["loss"],label="train_loss")
plt.plot(history.history["val_loss"],label="validation_loss")
plt.legend()
plt.show()

## Use the model

In [0]:
print(X_train3.shape)

In [0]:
n = 35
input_image = X_train3[n][np.newaxis,...]
print("label is: ", y_train3[n])

predictions = model.predict(input_image)
print("prediction is",predictions[0])

See that we just input image and get the results, while step 1 and step 2 required to construct feature(s) by our self.
This seed classification task was quite easy, but the complex the image gets, the harder to construct the most suitable feature gets.
That is when CNN shows its power.
See also the Yeast GFP protein localization tutorial for more complex image classification task.